In [ ]:
build = 'archive' # 'archive' uses the neurolibre archive of the data., 'latest' would download the latest versions of the data.

<head>

<link rel="preconnect" href="https://fonts.googleapis.com">
<link rel="preconnect" href="https://fonts.gstatic.com" crossorigin>
<link href="https://fonts.googleapis.com/css2?family=STIX+Two+Text:ital,wght@0,400;0,700;1,400&display=swap" rel="stylesheet">
</head>


<p>
<center>
<b>
<h3>
RF Shimming 7T
</h3>
</b>
</center>


# 1 &nbsp; &nbsp; | &nbsp; &nbsp; INTRODUCTION


Test citation: {cite:p}`DAstous2023`.

# References 

```{bibliography}
:filter: docname in docnames
```